In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# load dataset

In [ ]:
data = pd.read_excel("/content/Data_Train.xlsx")

# Data pre-processing

In [ ]:
data.head()

In [ ]:
data.shape

Checking null values

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(inplace=True)

In [ ]:
data[["Dep_hr","Dep_min"]] = data["Dep_Time"].str.split(":",expand=True)

In [ ]:
data.head(5)

In [ ]:
data = data.drop(columns=["Date_of_Journey","Dep_Time","Route"],axis=0)

In [ ]:
data.head(5)

In [ ]:
data["Arrival_Time"]=pd.to_datetime(data["Arrival_Time"])

In [ ]:
data["arr_hour"]=data["Arrival_Time"].dt.hour
data["arr_min"]=data["Arrival_Time"].dt.minute

In [ ]:
data=data.drop("Arrival_Time",axis=1)

In [ ]:
data.head(2)

In [ ]:
duration_hours = [int(x.split('h')[0].strip()) if 'h' in x else 0 for x in
 data['Duration']]
duration_mins = [int(x.split('m')[0].split()[-1].strip()) if 'm' in x else 0
for x in data['Duration']]

In [ ]:
data['Duration_hours']=duration_hours
data['Duration_mins']=duration_mins


In [ ]:
data.head(2)

In [ ]:
data=data.drop(columns=["Duration","Additional_Info"],axis=1)

In [ ]:
data.head(2)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
data.replace({'non-stop':0, '1 stop':1, '2 stops':2, '3 stops':3, '4 stops':4},inplace=True)

In [ ]:
#data["Total_Stops"]=le.fit_transform(data["Total_Stops"])

In [ ]:
data["Total_Stops"].value_counts()

In [ ]:
data.head(2)

In [ ]:
a = pd.get_dummies(data=data,columns=["Airline","Source","Destination"],drop_first=True)

In [ ]:
a.head(2)

In [ ]:
a.shape

In [ ]:
x=a.drop("Price",axis=1)
y=a["Price"]

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(a.corr(),annot=True)
plt.show()

# future scaling

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gb=GradientBoostingRegressor()

In [ ]:
gb.fit(x,y)
print(gb.feature_importances_)

In [ ]:
plt.figure(figsize=(12,8))
model=pd.Series(model.feature_importances_,index=x.columns)
model.nlargest(20).plot(kind='barh')
plt.show()

# spliting the data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

# Training model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor()
model.fit(X_train,y_train)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
model.score(X_train,y_train)

In [ ]:
model.score(X_test,y_test)


In [ ]:
prediction=model.predict(X_test)

In [ ]:
from sklearn import metrics
print(metrics.r2_score(y_test,prediction))

In [ ]:
n_estimators=[int(x) for x in np.linspace(start=100, stop=1200, num=12)]
max_features=['auto','sqrt']
max_depth=[int(x) for x in np.linspace(5,30,num=6)]
min_samples_split=[2,5,10,15,100]
min_samples_leaf=[1,2,5,10]


In [ ]:
random_grid={
'n_estimators':n_estimators,
'max_features':max_features,
'max_depth':max_depth,
'min_samples_split':min_samples_split,
'min_samples_leaf':min_samples_leaf}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
model2=RandomizedSearchCV(estimator=model, param_distributions=random_grid,scoring='neg_mean_squared_error',n_iter=10, cv=4,random_state=42,n_jobs=1)


In [ ]:
model2.get_params()

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)
